In [ ]:
import feedparser

In [ ]:
rss_url = 'https://rss.nytimes.com/services/xml/rss/nyt/AsiaPacific.xml'

In [ ]:
feed = feedparser.parse(rss_url)

In [ ]:
print(f"Feed Title: {feed.feed.title}")
print(f"Feed Description: {feed.feed.description}")

In [ ]:
for entry in feed.entries[:5]:
    print("Title:", entry.title)
    print("Link:", entry.link)
    print("Published:", entry.published)
    print("Summary:", entry.summary)
    print("-" * 80)

In [ ]:
output = {
    'id':feed.entries[0].id,
    'title':feed.entries[0].title,
    'url':feed.entries[0].link,
    'author':feed.entries[0].author,
    'date':feed.entries[0].published
}

In [ ]:
output

In [ ]:
feed.entries[0]